In [1]:
import sys

import pandas as pd
import numpy as np
#import matplotlib as plt
#from matplotlib import pyplot
#import scipy as sp
#from xgboost import XGBClassifier
import sklearn
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn import svm

from sklearn.metrics import mean_squared_error, r2_score

from sklearn import preprocessing

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
filename = 'ginf.csv'
games = pd.read_csv('./data/' + filename)
selected = games[games.odd_over.notnull() & games.odd_under.notnull()].copy()
selected

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
9135,d4i30ksD/,/soccer/france/ligue-1/bastia-paris-sg-d4i30ksD/,True,2016-08-12,F1,2017,france,Bastia,Paris Saint-Germain,0,1,12.13,5.50,1.38,1.92,2.12,2.25,1.74
9136,OAqkOhZJ/,/soccer/france/ligue-1/monaco-guingamp-OAqkOhZJ/,True,2016-08-12,F1,2017,france,AS Monaco,Guingamp,2,2,1.77,3.85,6.20,2.14,1.83,2.05,1.93
9137,21rgNClQ/,/soccer/france/ligue-1/caen-lorient-21rgNClQ/,True,2016-08-13,F1,2017,france,Caen,Lorient,3,2,2.46,3.32,3.50,2.28,1.78,1.95,2.02
9138,4j0rLEUb/,/soccer/england/premier-league/southampton-wat...,True,2016-08-13,E0,2017,england,Southampton,Watford,1,1,1.84,3.85,5.10,2.15,1.84,2.05,1.93
9139,6Fbj3icf/,/soccer/france/ligue-1/montpellier-angers-6Fbj...,True,2016-08-13,F1,2017,france,Montpellier,Angers,1,0,1.95,3.36,5.50,2.80,1.53,2.40,1.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10107,xAkY8l6R/,/soccer/italy/serie-a/genoa-crotone-xAkY8l6R/,True,2017-01-22,I1,2017,italy,Genoa,Crotone,2,2,1.97,4.35,8.00,1.95,2.03,2.03,1.86
10108,xSU9scI9/,/soccer/england/premier-league/chelsea-hull-ci...,True,2017-01-22,E0,2017,england,Chelsea,Hull,2,0,1.19,8.50,20.00,1.54,2.68,2.40,1.66
10109,xY7uZwOI/,/soccer/france/ligue-1/monaco-lorient-xY7uZwOI/,True,2017-01-22,F1,2017,france,AS Monaco,Lorient,4,0,1.32,6.24,11.50,1.53,3.08,1.80,2.25
10110,YyeGxMX8/,/soccer/spain/laliga/betis-gijon-YyeGxMX8/,True,2017-01-22,SP1,2017,spain,Real Betis,Sporting Gijon,0,0,1.74,4.07,5.90,2.20,1.89,2.05,1.86


In [3]:
selected['total'] = selected['fthg'] + selected['ftag']

col = 'total'
conditions = [selected[col] < 2.5 , selected[col] > 2.5]
choices = ['under','over']
selected['total'] = np.select(conditions,choices,default=np.nan)
selected.head()

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n,total
9135,d4i30ksD/,/soccer/france/ligue-1/bastia-paris-sg-d4i30ksD/,True,2016-08-12,F1,2017,france,Bastia,Paris Saint-Germain,0,1,12.13,5.50,1.38,1.92,2.12,2.25,1.74,under
9136,OAqkOhZJ/,/soccer/france/ligue-1/monaco-guingamp-OAqkOhZJ/,True,2016-08-12,F1,2017,france,AS Monaco,Guingamp,2,2,1.77,3.85,6.20,2.14,1.83,2.05,1.93,over
9137,21rgNClQ/,/soccer/france/ligue-1/caen-lorient-21rgNClQ/,True,2016-08-13,F1,2017,france,Caen,Lorient,3,2,2.46,3.32,3.50,2.28,1.78,1.95,2.02,over
9138,4j0rLEUb/,/soccer/england/premier-league/southampton-wat...,True,2016-08-13,E0,2017,england,Southampton,Watford,1,1,1.84,3.85,5.10,2.15,1.84,2.05,1.93,under
9139,6Fbj3icf/,/soccer/france/ligue-1/montpellier-angers-6Fbj...,True,2016-08-13,F1,2017,france,Montpellier,Angers,1,0,1.95,3.36,5.50,2.80,1.53,2.40,1.70,under


In [4]:
temp = selected.iloc[:,[-1,5,7,8,14,15,16,17]]
#dummies = pd.get_dummies(temp, columns=['ht', 'at','season'])
temp

,total,season,ht,at,odd_over,odd_under,odd_bts,odd_bts_n
9135,under,2017,Bastia,Paris Saint-Germain,1.92,2.12,2.25,1.74
9136,over,2017,AS Monaco,Guingamp,2.14,1.83,2.05,1.93
9137,over,2017,Caen,Lorient,2.28,1.78,1.95,2.02
9138,under,2017,Southampton,Watford,2.15,1.84,2.05,1.93
9139,under,2017,Montpellier,Angers,2.80,1.53,2.40,1.70
...,...,...,...,...,...,...,...,...
10107,over,2017,Genoa,Crotone,1.95,2.03,2.03,1.86
10108,under,2017,Chelsea,Hull,1.54,2.68,2.40,1.66
10109,over,2017,AS Monaco,Lorient,1.53,3.08,1.80,2.25
10110,under,2017,Real Betis,Sporting Gijon,2.20,1.89,2.05,1.86


In [12]:
data = pd.DataFrame();
data['total'] = 0
for i, j in temp.iterrows(): 
    if j['ht'] not in data.columns:
            data[j['ht']] = 0
    if j['at'] not in data.columns:
        data[j['at']] = 0
    if j['season'] not in data.columns:
        data[j['season']] = 0
    data = data.append({'total': j['total'], j['season']: 1, j['ht'] : 1 , 
                        j['at']: -1 , 'odd_under' : j['odd_under'], 'odd_over' : j['odd_over'],
                       'odd_bts' : j['odd_bts'], 'odd_bts_n' : j['odd_bts_n']}, ignore_index=True)

data


,total,Bastia,Paris Saint-Germain,2017,odd_bts,odd_bts_n,odd_over,odd_under,AS Monaco,Guingamp,...,FC Augsburg,VfL Wolfsburg,Borussia Monchengladbach,Bayer Leverkusen,Eintracht Frankfurt,Schalke 04,Hertha Berlin,SC Freiburg,TSG Hoffenheim,RB Leipzig
0,under,1.0,-1.0,1,2.25,1.74,1.92,2.12,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,over,NaN,NaN,1,2.05,1.93,2.14,1.83,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,over,NaN,NaN,1,1.95,2.02,2.28,1.78,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,under,NaN,NaN,1,2.05,1.93,2.15,1.84,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,under,NaN,NaN,1,2.40,1.70,2.80,1.53,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,over,NaN,NaN,1,2.03,1.86,1.95,2.03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
973,under,NaN,NaN,1,2.40,1.66,1.54,2.68,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
974,over,NaN,NaN,1,1.80,2.25,1.53,3.08,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
975,under,NaN,NaN,1,2.05,1.86,2.20,1.89,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
data = data.replace(np.nan, 0)
X = data.iloc[:,1:]
y = data.iloc[:,:1]
y = np.ravel(y)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1)
y

(977, 103)
(977,)


array(['under', 'over', 'over', 'under', 'under', 'over', 'over', 'under',
       'over', 'under', 'over', 'under', 'under', 'under', 'over',
       'under', 'over', 'under', 'over', 'over', 'under', 'under', 'over',
       'under', 'under', 'under', 'under', 'under', 'over', 'over',
       'over', 'under', 'over', 'under', 'over', 'over', 'under', 'under',
       'over', 'under', 'over', 'over', 'under', 'over', 'under', 'over',
       'under', 'over', 'under', 'over', 'under', 'over', 'over', 'over',
       'over', 'over', 'under', 'over', 'over', 'under', 'under', 'over',
       'over', 'under', 'over', 'under', 'under', 'over', 'under', 'over',
       'under', 'under', 'under', 'under', 'under', 'under', 'under',
       'over', 'under', 'over', 'over', 'under', 'under', 'over', 'under',
       'under', 'under', 'under', 'over', 'under', 'over', 'under',
       'under', 'over', 'under', 'over', 'over', 'over', 'over', 'over',
       'under', 'over', 'under', 'under', 'over', 'under'

In [14]:
pipeline = make_pipeline(KNeighborsClassifier())

hyperparameters = { 'kneighborsclassifier__n_neighbors' : [15, 40, 100],
                  'kneighborsclassifier__algorithm': ['auto', 'ball_tree', 'kd_tree'],
                  'kneighborsclassifier__weights': ['uniform','distance']}


clf = GridSearchCV(pipeline, hyperparameters, cv=10)

In [15]:
#clf = RandomForestClassifier(n_estimators=200)
#clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(30,20,3), random_state=1 , max_iter = 1000)
clf = KNeighborsClassifier(n_neighbors=91, algorithm='auto',weights='uniform');
#clf = RandomForestClassifier(n_estimators=400)
#clf = svm.SVC(kernel='linear')
clf.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=91, p=2,
                     weights='uniform')

In [16]:
accuracy = clf.score(X_test, y_test)
#y_pred = clf.predict_proba(X_test)
#predict = clf.predict(X_test)
#y_total = y_train.count()
#y_positive = y_train.sum()
#print('The training set contains {} examples (games) of which {} are positives .'.format(y_total, y_positive))
print('The accuracy of classifying is {:.2f} %'.format(accuracy*100))

The accuracy of classifying is 58.77 %
